In [3]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [4]:
flights = pd.read_pickle('flights.pkl')

In [5]:
weather = pd.read_csv('weather.csv')
weather.head()

,cloud_cover,precipitation,snow_depth,snow_fall,wind_speed,airport,name,new_date
0,46,0.0,0.0,0.0,2.8,DEN,DEN airport,2018-01-01
1,6,0.0,0.0,0.0,4.6,DEN,DEN airport,2018-01-02
2,14,0.0,0.0,0.0,6.0,DEN,DEN airport,2018-01-03
3,5,0.0,0.0,0.0,5.0,DEN,DEN airport,2018-01-04
4,36,0.0,0.0,0.0,4.2,DEN,DEN airport,2018-01-05


In [6]:
flights = pd.merge(flights, weather,  how='left', left_on=['fl_date','origin'], right_on = ['new_date','airport'])


In [7]:
flights.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,longest_add_gtime,no_name,cloud_cover,precipitation,snow_depth,snow_fall,wind_speed,airport,name,new_date
0,2018-09-30,WN,WN,WN,6696,WN,N8577Z,6696,11292,DEN,...,NaN,NaN,42.0,0.0,0.0,0.0,3.7,DEN,DEN airport,2018-09-30
1,2018-09-30,WN,WN,WN,103,WN,N8518R,103,11292,DEN,...,NaN,NaN,42.0,0.0,0.0,0.0,3.7,DEN,DEN airport,2018-09-30
2,2018-09-30,WN,WN,WN,2908,WN,N499WN,2908,11292,DEN,...,NaN,NaN,42.0,0.0,0.0,0.0,3.7,DEN,DEN airport,2018-09-30
3,2018-09-30,WN,WN,WN,3216,WN,N8669B,3216,11292,DEN,...,NaN,NaN,42.0,0.0,0.0,0.0,3.7,DEN,DEN airport,2018-09-30
4,2018-09-30,WN,WN,WN,3705,WN,N8329B,3705,11292,DEN,...,NaN,NaN,42.0,0.0,0.0,0.0,3.7,DEN,DEN airport,2018-09-30


In [8]:
# drop no_name column, it is all NaN.  Drop redundant airport and date columns
flights.drop(['no_name','airport','new_date','name'],axis=1,inplace=True)

In [9]:
# drop rows without weather data
flights = flights[flights['cloud_cover'].isna()==False]
flights = flights[flights['snow_fall'].isna()==False]


In [10]:
# consolidate delay codes into single column
conditions = [
    (flights['carrier_delay'] > 0),
    (flights['weather_delay'] > 0),
    (flights['nas_delay'] > 0),
    (flights['security_delay'] > 0),
    (flights['late_aircraft_delay'] > 0),
    (flights['arr_delay'] <=0)
    ]

# create a list of the values we want to assign for each code
values = [1,2,3,4,5,0]

# create a new column and use np.select to assign values to it using our lists as arguments
flights['delay_code'] = np.select(conditions, values)


In [11]:
# remove the delay type columns
flights.drop(['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay'],axis=1,inplace=True)

# remove gtime columns as these are only calculated after a flight has already been cancelled
flights.drop(['total_add_gtime','longest_add_gtime'],axis=1,inplace=True)

# remove mkt_unique_carrier, branded_code_share, mkt_carrier, mkt_carrier_fl_num as these are redundant
flights.drop(['mkt_unique_carrier','branded_code_share','mkt_carrier','mkt_carrier_fl_num'],axis=1,inplace=True)

# remove tail_num, origin_city_name, origin_airport_id, dest_airport_id, dest_city_name. Redundant info
flights.drop(['tail_num','origin_city_name','dest_city_name'],axis=1,inplace=True)

#remove first_dep_time, nearly all NaN
flights.drop('first_dep_time',axis=1, inplace=True)

#remove dup, all same value
flights.drop('dup',axis=1, inplace=True)

# remove flight number
flights.drop('op_carrier_fl_num',axis=1,inplace=True)

In [12]:
# replace NaN values with 0 for cancellation code, actual_elapsed_time, air_time, crs_elapsed_time

flights['cancellation_code'].fillna(0,inplace=True)
flights['actual_elapsed_time'].fillna(0,inplace=True)
flights['air_time'].fillna(0,inplace=True)
flights['crs_elapsed_time'].fillna(0,inplace=True)

In [13]:
#change date to datetime instead of object
flights['fl_date'] = flights['fl_date'].astype('datetime64[ns]')

# change dep_time and arr_time to binned groups, every 4 hours
bins = [0, 400, 800, 1200, 1600, 2000, 2400]
labels = [1,2,3,4,5,6]

flights['crs_arr_binned'] = pd.cut(flights['crs_arr_time'], bins=bins, labels=labels)
flights['crs_drp_binned'] = pd.cut(flights['crs_dep_time'], bins=bins, labels=labels)

#drop former dep and arr time columns
flights.drop(['crs_dep_time','crs_arr_time'],axis=1, inplace=True)

In [14]:
# add columns to capture year, month, date, day of week
flights['year'] = flights.fl_date.dt.year
flights['month'] = flights.fl_date.dt.month
flights['day'] = flights.fl_date.dt.day
flights['day_of_week'] = flights.fl_date.dt.dayofweek


In [15]:
# bin distance into short, medium, long haul
"""
Short Haul - Distance <800 miles
Medium Haul - Distance 800-2200 miles
Long Haul - Distance >2200miles
"""

dist_bins = [0, 800, 2200, 40000]
dist_labels = [1,2,3]

flights['dist_binned'] = pd.cut(flights['distance'], bins=dist_bins, labels=dist_labels)

# remove distance column
flights.drop('distance',axis=1, inplace=True)

In [16]:
# bin delays
"""
Large airlines must pay:
$400 if the passenger arrives three hours or more hours late, but less than six hours;
$700 if the passenger arrives six or more hours late, but less than nine hours; and
$1,000 if the passenger arrives nine or more hours late.
"""

delay_bins = [-3000, 0, 30, 60, 180, 360, 540, 5000]
delay_labels = [0,1,2,3,4,5,6]

flights['dep_delay_bin'] = pd.cut(flights['dep_delay'], bins=delay_bins, labels=delay_labels)
flights['arr_delay_bin'] = pd.cut(flights['arr_delay'], bins=delay_bins, labels=delay_labels)

# remove distance column
#flights.drop(['dep_delay','arr_delay'],axis=1, inplace=True)


In [17]:
# add in average passengers for airport and carrier
avg_airport_pass = pd.read_csv('avg_airport_passengers.csv')
avg_carrier_pass = pd.read_csv('avg_carrier_passengers.csv')

flights = pd.merge(flights,avg_airport_pass,how='left', left_on='origin_airport_id', right_on='airline_id')
flights = pd.merge(flights,avg_carrier_pass,how='left', left_on='op_unique_carrier', right_on='unique_carrier')

In [18]:
# add in carrier and airport average delays
flights['car_avg_delay'] = flights.groupby(['op_unique_carrier'])['dep_delay'].transform('mean')
flights['origin_avg_delay'] = flights.groupby(['origin'])['dep_delay'].transform('mean')
flights['dest_avg_delay'] = flights.groupby(['dest'])['dep_delay'].transform('mean')

In [19]:
flights = flights.sort_values(by=['origin','op_unique_carrier','fl_date'])

In [20]:
# add in moving averages by airport
flights['airport_dep_delay_moving_2'] = flights.groupby(['origin','fl_date'])['dep_delay'].transform(lambda x: x.rolling(2, 1).mean())
flights['airport_dep_delay_moving_5'] = flights.groupby(['origin','fl_date'])['dep_delay'].transform(lambda x: x.rolling(5, 1).mean())
flights['airport_arr_delay_moving_2'] = flights.groupby(['dest','fl_date'])['arr_delay'].transform(lambda x: x.rolling(2, 1).mean())
flights['airport_arr_delay_moving_5'] = flights.groupby(['dest','fl_date'])['arr_delay'].transform(lambda x: x.rolling(5, 1).mean())


In [21]:
# add in moving averages by carrier
flights['carrier_dep_delay_moving_2'] = flights.groupby(['op_unique_carrier','fl_date'])['dep_delay'].transform(lambda x: x.rolling(2, 1).mean())
flights['carrier_dep_delay_moving_5'] = flights.groupby(['op_unique_carrier','fl_date'])['dep_delay'].transform(lambda x: x.rolling(5, 1).mean())
flights['carrier_arr_delay_moving_2'] = flights.groupby(['op_unique_carrier','fl_date'])['arr_delay'].transform(lambda x: x.rolling(2, 1).mean())
flights['carrier_arr_delay_moving_5'] = flights.groupby(['op_unique_carrier','fl_date'])['arr_delay'].transform(lambda x: x.rolling(5, 1).mean())


In [22]:
# replace strings in cancellation code column
code_list = flights.cancellation_code.unique().tolist()
code_label = list(range(1,7))
flights['cancellation_code'].replace(code_list,code_label,inplace=True)

In [23]:
# drop redundant carrier label
flights.drop('unique_carrier',axis=1, inplace=True)

# drop reduncant airline id and passengers column
flights.drop(['airline_id','passengers_x'],axis=1, inplace=True)

In [24]:
# replace nan values in moving average with 0
flights['airport_dep_delay_moving_2'] = flights['airport_dep_delay_moving_2'].fillna(0)
flights['airport_dep_delay_moving_5'] = flights['airport_dep_delay_moving_5'].fillna(0)
flights['airport_arr_delay_moving_2'] = flights['airport_arr_delay_moving_2'].fillna(0)
flights['airport_arr_delay_moving_5'] = flights['airport_arr_delay_moving_5'].fillna(0)
flights['carrier_dep_delay_moving_2'] = flights['carrier_dep_delay_moving_2'].fillna(0)
flights['carrier_dep_delay_moving_5'] = flights['carrier_dep_delay_moving_5'].fillna(0)
flights['carrier_arr_delay_moving_2'] = flights['carrier_arr_delay_moving_2'].fillna(0)
flights['carrier_arr_delay_moving_5'] = flights['carrier_arr_delay_moving_5'].fillna(0)

In [25]:
# change cloud cover to decimal instead of percentage
flights['cloud_cover'] = flights['cloud_cover'] / 100

In [26]:
# change carrier information to values
carrier_list = flights['op_unique_carrier'].unique().tolist()
carrier_label = list(range(1,29))
carrier_dict = dict(zip(carrier_list,carrier_label))
flights['op_unique_carrier'].replace(carrier_list,carrier_label, inplace=True)

#drop origin and dest as it is redundant to airline id
flights.drop('origin',axis=1, inplace=True)
flights.drop('dest',axis=1, inplace=True)

In [27]:
flights.to_pickle('flights_full.pkl')

In [28]:
flights_copy = deepcopy(flights)

In [29]:
# create two separate dataframes, one to handle delays prediction, one to handle cancellation prediction
delay_df = deepcopy(flights)
cancel_df = deepcopy(flights)

In [30]:
# remove info for once the plane is in the air, not helpful in predicting
delay_df.drop(['crs_elapsed_time','air_time','actual_elapsed_time'],axis=1,inplace=True)

# for delays_df, remove cancellation, diverted columns and cancellation_code
delay_df.drop(['cancelled','diverted','cancellation_code'],axis=1,inplace=True)

# remove arr_time, dep_time,taxi and wheels on/off info, I don't believe this is actually useful
delay_df.drop(['dep_time','taxi_out','wheels_off','wheels_on','taxi_in','arr_time'],axis=1,inplace=True)

In [31]:
# remove any leftover rows with NaN values on dep_delay and arr_delay
delay_df = delay_df[(delay_df['dep_delay_bin'].isna()==False)
                   & (delay_df['arr_delay_bin'].isna()==False)]


In [32]:
print('Dataframe Size:',delay_df.shape)
print(delay_df.isna().sum())
delay_df.head(10)

Dataframe Size: (15771466, 34)
fl_date                       0
op_unique_carrier             0
origin_airport_id             0
dest_airport_id               0
dep_delay                     0
arr_delay                     0
flights                       0
cloud_cover                   0
precipitation                 0
snow_depth                    0
snow_fall                     0
wind_speed                    0
delay_code                    0
crs_arr_binned                0
crs_drp_binned                0
year                          0
month                         0
day                           0
day_of_week                   0
dist_binned                   0
dep_delay_bin                 0
arr_delay_bin                 0
passengers_y                  0
car_avg_delay                 0
origin_avg_delay              0
dest_avg_delay                0
airport_dep_delay_moving_2    0
airport_dep_delay_moving_5    0
airport_arr_delay_moving_2    0
airport_arr_delay_moving_5    0
carrier_d

,fl_date,op_unique_carrier,origin_airport_id,dest_airport_id,dep_delay,arr_delay,flights,cloud_cover,precipitation,snow_depth,...,origin_avg_delay,dest_avg_delay,airport_dep_delay_moving_2,airport_dep_delay_moving_5,airport_arr_delay_moving_2,airport_arr_delay_moving_5,carrier_dep_delay_moving_2,carrier_dep_delay_moving_5,carrier_arr_delay_moving_2,carrier_arr_delay_moving_5
14967504,2018-02-05,1,10135,10397,3.0,-1.0,1,0.29,0.00,0.7,...,10.58383,7.743845,3.0,3.0,-1.0,-1.0,3.0,3.0,-1.0,-1.0
14983724,2018-02-06,1,10135,10397,-14.0,-8.0,1,0.38,0.00,0.3,...,10.58383,7.743845,-14.0,-14.0,-8.0,-8.0,-14.0,-14.0,-8.0,-8.0
15000956,2018-02-07,1,10135,10397,-4.0,-2.0,1,0.89,0.39,3.1,...,10.58383,7.743845,-4.0,-4.0,-2.0,-2.0,-4.0,-4.0,-2.0,-2.0
15030386,2018-02-08,1,10135,10397,-6.0,-17.0,1,0.40,0.00,1.8,...,10.58383,7.743845,-6.0,-6.0,-17.0,-17.0,-6.0,-6.0,-17.0,-17.0
15048602,2018-02-09,1,10135,10397,9.0,8.0,1,0.74,0.00,1.1,...,10.58383,7.743845,9.0,9.0,8.0,8.0,9.0,9.0,8.0,8.0
15085591,2018-02-11,1,10135,10397,2.0,25.0,1,0.98,0.36,0.0,...,10.58383,7.743845,2.0,2.0,25.0,25.0,2.0,2.0,25.0,25.0
15110810,2018-02-12,1,10135,10397,-5.0,-18.0,1,0.42,0.00,0.0,...,10.58383,7.743845,-5.0,-5.0,-18.0,-18.0,-5.0,-5.0,-18.0,-18.0
15123146,2018-02-13,1,10135,10397,-7.0,17.0,1,0.31,0.00,0.0,...,10.58383,7.743845,-7.0,-7.0,17.0,17.0,-7.0,-7.0,17.0,17.0
15147793,2018-02-14,1,10135,10397,-12.0,-26.0,1,0.76,0.03,0.0,...,10.58383,7.743845,-12.0,-12.0,-26.0,-26.0,-12.0,-12.0,-26.0,-26.0
15166668,2018-02-15,1,10135,10397,-8.0,-24.0,1,0.73,0.15,0.0,...,10.58383,7.743845,-8.0,-8.0,-24.0,-24.0,-8.0,-8.0,-24.0,-24.0


In [33]:
# for cancel model, remove all columns that deal with delays
cancel_df.drop(['dep_time','dep_delay_bin','taxi_out','wheels_off','wheels_on','taxi_in','arr_time','arr_delay_bin','delay_code'],axis=1,inplace=True)

In [34]:
# remove variables that have to do when the flight is in the air
cancel_df.drop(['crs_elapsed_time','actual_elapsed_time','air_time'],axis=1,inplace=True)

In [38]:
cancel_df.drop(['dep_delay','arr_delay'],axis=1,inplace=True)

In [35]:
print(cancel_df.shape)
cancel_df.dtypes

(16090971, 34)


fl_date                       datetime64[ns]
op_unique_carrier                      int64
origin_airport_id                      int64
dest_airport_id                        int64
dep_delay                            float64
arr_delay                            float64
cancelled                              int64
cancellation_code                      int64
diverted                               int64
flights                                int64
cloud_cover                          float64
precipitation                        float64
snow_depth                           float64
snow_fall                            float64
wind_speed                           float64
crs_arr_binned                      category
crs_drp_binned                      category
year                                   int64
month                                  int64
day                                    int64
day_of_week                            int64
dist_binned                         category
passengers

In [36]:
print(delay_df.shape)
delay_df.head(10)

(15771466, 34)


,fl_date,op_unique_carrier,origin_airport_id,dest_airport_id,dep_delay,arr_delay,flights,cloud_cover,precipitation,snow_depth,...,origin_avg_delay,dest_avg_delay,airport_dep_delay_moving_2,airport_dep_delay_moving_5,airport_arr_delay_moving_2,airport_arr_delay_moving_5,carrier_dep_delay_moving_2,carrier_dep_delay_moving_5,carrier_arr_delay_moving_2,carrier_arr_delay_moving_5
14967504,2018-02-05,1,10135,10397,3.0,-1.0,1,0.29,0.00,0.7,...,10.58383,7.743845,3.0,3.0,-1.0,-1.0,3.0,3.0,-1.0,-1.0
14983724,2018-02-06,1,10135,10397,-14.0,-8.0,1,0.38,0.00,0.3,...,10.58383,7.743845,-14.0,-14.0,-8.0,-8.0,-14.0,-14.0,-8.0,-8.0
15000956,2018-02-07,1,10135,10397,-4.0,-2.0,1,0.89,0.39,3.1,...,10.58383,7.743845,-4.0,-4.0,-2.0,-2.0,-4.0,-4.0,-2.0,-2.0
15030386,2018-02-08,1,10135,10397,-6.0,-17.0,1,0.40,0.00,1.8,...,10.58383,7.743845,-6.0,-6.0,-17.0,-17.0,-6.0,-6.0,-17.0,-17.0
15048602,2018-02-09,1,10135,10397,9.0,8.0,1,0.74,0.00,1.1,...,10.58383,7.743845,9.0,9.0,8.0,8.0,9.0,9.0,8.0,8.0
15085591,2018-02-11,1,10135,10397,2.0,25.0,1,0.98,0.36,0.0,...,10.58383,7.743845,2.0,2.0,25.0,25.0,2.0,2.0,25.0,25.0
15110810,2018-02-12,1,10135,10397,-5.0,-18.0,1,0.42,0.00,0.0,...,10.58383,7.743845,-5.0,-5.0,-18.0,-18.0,-5.0,-5.0,-18.0,-18.0
15123146,2018-02-13,1,10135,10397,-7.0,17.0,1,0.31,0.00,0.0,...,10.58383,7.743845,-7.0,-7.0,17.0,17.0,-7.0,-7.0,17.0,17.0
15147793,2018-02-14,1,10135,10397,-12.0,-26.0,1,0.76,0.03,0.0,...,10.58383,7.743845,-12.0,-12.0,-26.0,-26.0,-12.0,-12.0,-26.0,-26.0
15166668,2018-02-15,1,10135,10397,-8.0,-24.0,1,0.73,0.15,0.0,...,10.58383,7.743845,-8.0,-8.0,-24.0,-24.0,-8.0,-8.0,-24.0,-24.0


In [39]:
# export dataframes as pickle to use in other notebooks
delay_df.to_pickle('delay_df.pkl')
cancel_df.to_pickle('cancel_df.pkl')